# CIS 5450 Homework 4: Machine Learning

<ins>Due Date</ins>: **March 24th at 11:59PM EST** \\
<ins>Total Points</ins>: **95 points** (= 84 autograded + 11 manually graded)

**PLEASE READ THE FAQ as you do this assignment!** It's pinned on Ed and we TAs work really hard to keep it updated with everything you might need to know or anything we might have failed to specify. Writing these HWs and test cases gets tricky since students always end up implementing solutions that we did not anticipate and thus could not have prepared the grader correctly for.

# Imports/Setup
Run the following cells to set up the notebook.


Please make sure you enter your **8 digit Penn ID** in the  student ID field below.

In [ ]:
%%capture
!pip install -i https://test.pypi.org/simple/ penn-grader==0.5.0
from penngrader.grader import *

# PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY.
# IF NOT, THE AUTOGRADER WON'T KNOW WHO TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID =                                                                    # YOUR PENN-ID GOES HERE AS AN INTEGER

In [ ]:
%%writefile config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
grader = PennGrader('config.yaml', 'cis5450o_spr25_HW4', STUDENT_ID, STUDENT_ID)

In [ ]:
# import packages
import json
import glob
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns

In [ ]:
%%capture
!apt update
!pip install kaggle

# **Part I:** Preprocessing and Modeling in `scikit-learn` (65 points)

## **1.1** Data Loading and Preprocessing [Total: 2 points]

### **1.1.1** Read Data

We are using one CSV for this homework, `Hotel Reservations.csv` from a Kaggle [dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download). The dataset contains a column called `booking_status`, a binary variable indicating whether a reservation was canceled or not.

To get the data in here:
1. Go to this [Kaggle link](https://www.kaggle.com) and create a Kaggle account (unless you already have one)
2. Go to Account and click on "Create New API Token" to get the API key in the form of a json file `kaggle.json`
3. Upload the `kaggle.json` file to the default location in your Google Drive, 'MyDrive' (Please **DO NOT** upload the json file into any _specific_ folder as it will be difficult for us to debug issues if you deviate from these instructions!).

This can be helpful for your project if you decide to use Kaggle.
Run the following cells to allow Colab to connect to Kaggle and to download the data used in this homework.

In [ ]:
# Run this cell to mount your drive (you will be prompted to sign in)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create the kaggle directory and
# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle

In [ ]:
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
# Download dataset
!!kaggle datasets download -d ahsan81/hotel-reservations-classification-dataset

In [ ]:
# Unzip folder in Colab content folder
!unzip /content/hotel-reservations-classification-dataset.zip

In [ ]:
# TO-DO: Read the csv file and save it to a dataframe called "df_reservations"


In [ ]:
# CHECK: Peak at the first five rows


### **1.1.2** Check Nulls and Duplicates

We will find the number of rows with null values and the number of duplicated rows.

Store the results into `num_nulls` and `num_dups`, respectively.

**Please use pandas functions to compute these values instead of hardcoding them.**

In [ ]:
# TO-DO: find number of rows with null values
num_nulls =

In [ ]:
# TO-DO: find number of duplicated rows
num_dups =

In [ ]:
#Grader Cell (2 points)
grader.grade(test_case_id = 'drop_null_and_dups', answer = (num_nulls, num_dups))

## **1.2** EDA [Total: 16 points]

Exploratory Data Analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modeling or hypothesis testing task.


### **1.2.1** Understanding Data

A good practice before approaching any data science problem, is to understand the data you will be working with. This can be through descriptive statistics, datatypes, or just a quick tabular visualization. We will be walking through such tasks through Pandas.

While not explicitly graded, if you don't do this, you are more likely to make mistakes down the line if you don't have a good understanding of your datasets and their range of values.

In [ ]:
# Display the datatypes in `df_reservations`


In [ ]:
# Display the descriptive statistics of `df_reservations`


We can do further visualization in the form of plots and graphs if we want to explore the data further, but in the case of this homework we stop now

### **1.2.2** Correlation of Feature Variables

With multiple features, it can be somewhat exhausting to do bivariate analysis on every possible pair of features. While you certainly should, your first instinct should be to check for the correlation between features since certain models (e.g. Linear Regression) will not work well if we have strong multicollinearity.

_**Isolating Numerics from Categorical Features**_

Before anything else, it may help to create groups of the numeric and categorical variables.

**Task:** is to split the `df_reservations` dataframe into 2 dataframes:

1. `numerics_df`: This dataframe contains all numerical columns from `df_reservations`

2. `categorical_df`: This dataframe contains all categorical columns from `df_reservations`
  - i.e. the columns of non-numeric type or contain binary values

In the above dataframes, **please do not include `arrival_year`, `arrival_month`, `arrival_date`, `Booking_ID`, `booking_status`**.
This is because:

* For the sake of this exercise we will assume that time related features ( `arrival_year`, `arrival_month`, `arrival_date`) can not be categorized as numeric or categorical
* We will use `booking_status` as the target variable for classification
* `Booking_ID` is just an identifier and does not hold any predictive information.

Also, **sort the columns of `numerics_df` and `categorical_df` in alphabetic order**

**Notes:**

Please use 'numerics_df' and 'categorical_df' as your dataframe names so that the autograder can recognize them.

Internal Server Error occurs when your output crashes the test case in the autograder.

**Hint:** Do not just automatically assume that if something is an integer it should be part of numerics_df.
For example if something has a yes/no answer (binary random variable) such as a column indicating whether someone is female or not female, this is technically categorical data!

In [ ]:
# TO-DO: Visualize number of unique values and datatype in each column (call .nunique())


In [ ]:
# TO-DO: Form 'numerics_df' and 'categorical_df'


In [ ]:
# TO-DO: sort the columns of `numerics_df` and `categorical_df` in alphabetic order


In [ ]:
#Grader Cell (3 points)
grader.grade(test_case_id = 'cat_num_df', answer = (numerics_df, categorical_df))

#### (b) **Correlation Heatmap** [3 pts]

**Manually Graded Task:** Create a correlation matrix using `numerics_df` and call it `corr_mat`. Using the correlation matrix, generate a correlation heatmap for these numeric features. We will be using Seaborn library to create this heatmap.

Make sure your correlation heatmap meets the following criteria:
*   Ensure that your heatmap is of figure size `(8,8)`: all feature labels should be visible on both the $x$-axis and $y$-axis
*   Use the `RdBu` color map to ensure that negative correlations are red and positive correlations are blue. This is far more intuitive than other ones.
*   Standardize the color scale so that -1 takes the darkest red color, 0 is totally white, and +1 takes the darkest blue color. Remember, the Official Documentation should always be your first point of reference.

In [ ]:
# TO-DO: Plot correlation heatmap (3 points)


## **1.3** Feature Engineering [Total: 4 points]

Feature engineering is the process of applying domain knowledge to extract features from raw data via data mining techniques. These features can be used to improve the performance of machine learning algorithms.

### **1.3.1** One Hot Encoding [2 pts]

One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction. With one-hot encoding, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns.

One way to one-hot encode in Pandas is by using `pd.get_dummies()` . The documentation can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).

**Input:** Perform operations on `df_reservations`

**Task:**
- Use `pd.get_dummies()` to one hot encode the following columns:
  - `"type_of_meal_plan"`
  - `"room_type_reserved"`
  - `"market_segment_type"`
- Save your result in the dataframe `encoded_df_reservations`.

**Note:** Do <ins>NOT</ins> modify the column names. Use the default names `pd.get_dummies()` generates.

In [ ]:
# TO-DO: drop "Booking_ID" column from `df_reservations`


In [ ]:
# TO-DO: create dataframe 'encoded_df_reservations' that contains the appropriate one hot encoded columns


In [ ]:
# CHECK: display the first two rows of 'encoded_df_reservations'


In [ ]:
#Grader Cell (2 points)
grader.grade(test_case_id = 'one_hot_encoded', answer = encoded_df_reservations.head(10))

### **1.3.2** Change 'booking_status' to Boolean [2 pts]

In [ ]:
# TO-DO: canceled = 1, not canceled = 0


In [ ]:
# Grader Cell (2 points)
grader.grade(test_case_id = 'boolean_booking', answer = encoded_df_reservations.head(20)['booking_status'])

## **1.4** Modeling (sklearn) [Total: 43 points]

### **1.4.1** Preprocessing



#### (a) Create Features and Label [4 pts]

Now that we have explored and cleaned our dataset, let's prepare it for a machine learning task. In this homework, you will work with various models and attempt to predict whether a room booking will be cancelled or not.

The features will be all the variables in the dataset **except** `"booking_status"`, which will act as the label for our problem. First, store these two as `features` (pd.DataFrame) and `target` (pd.Series), respectively.

In [ ]:
# TO-DO: stores features dataframe into variable called "features"


In [ ]:
# TO-DO: store the classification target variable into "target"


Now, use scikit-learn's `train_test_split` function to split data for regression into train and test sets. The split should be 80-20 meaning 80% for training and rest for testing.

**_IMPORTANT_:** Please set the `seed` variable to 42, then set the parameter to `random_state = seed`, and then finally store the resulting splits as `X_train, X_test, y_train,` and `y_test`.

In [ ]:
# TO-DO: Assign appropriate value to seed and conduct 80/20 train-test split with random_state = seed


In [ ]:
# Grader Cell (4 points)
grader.grade(test_case_id = 'train_test_split_classification', answer = (X_train.shape, X_test.shape,
                                                                         y_train.shape, y_test.shape,
                                                                         seed, Counter(y_test)))

### **1.4.2** Classification Models

#### (a) Logistic Regression [2 pts]

Fit a Logistic regression classifier on the `X_train` and `y_train` with the default hyperparameters. Calculate the accuracy of the model on the test set using the `score` method and store it in a variable named `log_acc`

If you receive warnings when training your model, you can ignore them for this homework.

**Note:**

Don't worry about convergence warnings for this HW.

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Initialize model with default parameters and fit it on the training set


# TO-DO: Use the model to predict on the test set and save these predictions as `y_pred`


# TO-DO: Find the accuracy and store the value in `log_acc`



In [ ]:
# Grader Cell [2 points]
grader.grade(test_case_id = 'check_log_clf', answer = log_acc)

#### (b) Random Forest Classifier [3 pts]

Fit a Random Forest classifier on the `X_train` and `y_train` with the following hyperparameters:
- balanced class_weight
- 120 estimators
- maximum depth of 30
- random seed set to 42

Calculate the accuracy of the model on the test set using the `score` method and store it in a variable named `rf_acc`.
Also, compute a [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) for your prediction and save it as `rf_confusion`.

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Initialize model with default parameters and fit it on the training set


# TO-DO: Use the model to predict on the test set and save these predictions as `y_pred`


# TO-DO: Find the accuracy and store the value in `rf_acc`


# TO-DO: Compute the confusion matrix and save it to `rf_confusion`


In [ ]:
# Grader Cell (3 points)
grader.grade(test_case_id = 'check_rf_clf', answer = (rf_acc, rf_confusion))

#### (c) Plot Feature Importances [5 Points]

Now that you have a Random Forest model, it's important for data scientists to understand and communicate to stakeholders the drivers that cause booking cancellations. **Feature importance** can help achieve this.

Here are the steps to follow:

* Use the built-in feature_importances_ function to obtain an array of feature importances.
* Identify the top 10 features by importance then create a DataFrame with two columns: one for the names of the current features (Called Feature) and another for their corresponding importance values (called Feature Importance).
* Visualize the top 10 feature importance using a bar plot.
* Analyze the top 10 features to determine if they make sense, and provide your reasoning (manual grading)

In [ ]:
# TO-DO: get feature importnce using built-in feature importance

# TO-DO: Get the top 10 and Create a DataFrame `feature_importance_df`

# TO-DO: Visualize the top 10 feature importance using a barplot

# TO-DO: Analyze the top 10 features to determine if they make sense, and provide your reasoning


In [ ]:
#Grader Cell
grader.grade(test_case_id = 'check_feature_importance', answer = (feature_importance_df.reset_index()))

#### (d) PCA to Reduce Dimensionality [4 + 2 pts]

The goal of Principal Component Analysis is to reduce number of dimensions of our features.

_**Initial PCA**_

As a first step, instantiate the `PCA` class from scikit-learn and fit it on your training set. We are not reducing the dimensionality of our data in this step but rather trying to find what would be the ideal number of Principal Components to choose. So keep all Principal Components for this step

Please remember that PCA is **not** scale-invariant! What does this imply you need to do first? This requires conceptual understanding to implement so make sure you _watched_ the corresponding lecture and recitation!

<ins>WARNING</ins>: It is _CRITICALLY_ important that you understand the difference between the `.fit_transform`, `.fit` and `.transform` methods. This has been the most common mistake students have made in this homework, which has potentially made it more difficult for them. Without fully understanding this, those same students have gone on to make the same mistakes in the final exam and their project. We cannot emphasize this enough!

In [ ]:
# TO-DO: Import necessary libraries



In [ ]:
# TO-DO: Intermediate step to address fac that PCA is not scale-invariant


# TO-DO: Instantiate and Fit PCA


_**Cumulative Explained Variance Ratios**_

Create an array of explained variance ratios and store it into a variable called `explained_variance_ratios`. Also, calculate the _cumulative_ explained variance ratios and store that into another variable called `cum_evr`.

In [ ]:
# TO-DO: Save the explained variance ratios into variable called "explained_variance_ratios"


# TO-DO: Save the CUMULATIVE explained variance ratios into variable called "cum_evr"


In [ ]:
#Grader cell (2 points)
grader.grade(test_case_id = 'check_pca_explained_variance', answer = (explained_variance_ratios, cum_evr))

Now plot the _cumulative_ `explained_variance_ratio` against the number of components to decide the number of components you should keep (this should look very similar to a visualization in lecture). Also add a horizontal line that represents the 80% of the variance as a threshold.

As before, you should ensure that the plot follows the best practices you've developed over the past 2 plotting exercises in this HW (Labelling the axes and adding title to the plot, readability, etc.). **This will be manually graded for 2 points**

_**CAUTION:** Recall that Python starts counting from 0, but this would make it very non-intuitive when reading from your graph. We strongly recommend that you modify the x-axis of your graph so that the **number of components begins at 1**. Otherwise, you may risk making silly mistakes in the subsequent sections._

In [ ]:
# TO-DO: find optimal num components to use (n) by plotting explained variance ratio (2 points)


_**Final PCA**_

Using your results above to help you decide the number of components to keep, choose a number (`n`) that explains **at least 80% of total variance** in the dataset. Then re-fit and transform your PCA instance on the training set using the number of components you decided.

Remember that your PCA instance should be **fit** on the training set (`X_train`) but **only transform and not fit** on the test set (`X_test`).

Call your transformed set of principal components `X_test_pca` in order to submit it to the autograder.

In [ ]:
# TO-DO: Get transformed set of principal components on x_test

# 1. Refit and transform on training with parameter n (as deduced from the last step)


# 2. Transform on Testing Set and store it as `X_test_pca`



In [ ]:
# Grader cell (2 points)
grader.grade(test_case_id = 'check_fitted_pca', answer = X_test_pca)

#### (e) Logistic Regression with PCA [4 pts]

1. Fit the logistic regression on your Final Principal Components data using your optimal `n`. Name the model `log_reg_pca`
2. Make predictions on the **test** set and store this as `y_pred`.
3. Report accuracy for the **test** set and call it `test_accuracy`

In [ ]:
# TO-DO: Initialize `log_reg_pca` model with default parameters and fit it on the PCA transformed training set


# TO-DO: Use the model to predict on the PCA transformed test set and save these predictions as `y_pred`


# TO-DO: Find the accuracy and store the value in `test_accuracy`


In [ ]:
# Grader (4 points)
grader.grade(test_case_id = 'check_log_reg_pca', answer = (test_accuracy, log_reg_pca.n_features_in_, y_pred))

### **1.4.3.0** Regression: Split Data into Train and Test [4 pts]


**Task:**
 We will be predicting `avg_price_per_room` for regression models.

 Drop the columns: `'arrival_year'`,`arrival_month`, `'arrival_date'`,`'no_of_previous_cancellations'`, `'no_of_previous_bookings_not_canceled'` `'booking_status'`
 from `encoded_df_reservations` and save the dataframe to `reg_df_reservations`. Then, use scikit-learn's `train_test_split` function to split data for regression into train and test sets. The split should be 80-20 meaning 80% for training and rest for testing.


**_IMPORTANT_: Please set the `seed` variable to 42 and then set the parameter to `random_state = seed`** and store the resulting splits as `X_train, X_test, y_train,` and `y_test`.


In [ ]:
# TO-DO: drop arrival_year, arrival_month, arrival_date, no_of_previous_cancellations, no_of_previous_bookings_not_canceled,booking_status


# TO-DO: save the dataframe to reg_df_reservations



In [ ]:
# TO-DO: stores features dataframe into variable called "features"


# TO-DO: store the classification target variable  (`avg_price_per_room`) into "target"



In [ ]:
# TO-DO: Assign appropriate value to seed and conduct 80/20 train-test split with random_state = seed


In [ ]:
# Grader cell (4 points)
grader.grade(test_case_id = 'train_test_split_regression', answer = (X_train.shape, X_test.shape,
                                                                     y_train.shape, y_test.shape,
                                                                     seed, len([i for i in y_test if i < 100])))

### **1.4.3.1** Regression Models

In this section, we will switch from classification models to regression models.

Let's use the features we created in 1.4.3.0 to create regression models and predict the **average price per room**.

#### (a) Linear Regression (Unregularized) [5 pts]

Use the `LinearRegression` class in scikit-learn to perform Linear Regression. Initialize a Linear regression model named `reg` with default parameters, fit the model to the training set, and then make predictions on the testing set.

Save your predictions in an array named `y_pred`, and report your R-squared score (saved it as a variable called `lin_reg_score`).

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Initialize model with default parameters and fit it on the training set


# TO-DO: Use the model to predict on the test set and save these predictions as `y_pred`


# TO-DO: Find the R-squared score and store the value in `lin_reg_score`


In [ ]:
# Grader (5 points)
grader.grade(test_case_id = 'check_linear_reg', answer = (lin_reg_score, reg.coef_, reg.intercept_))

#### (b) Ridge Regression [2 pts]

Use the `Ridge` class in scikit-learn to perform $L_2$ Regularized Linear Regression. Initialize a Ridge regression model named `reg_ridge` with regularization strength `alpha = 10`, fit the model to the training set, and then make predictions on the testing set.

**Note:** Recall that Ridge regression is not scale-invariant, so you will need to standardize the features prior to modeling.

Report your $R^2$ score and save it as a variable called `ridge_score`.

In [ ]:
# TO-DO: Intermediate step to address scale-invariance


In [ ]:
# TO-DO: Import required libraries


# TO-DO: Initialize model with alpha = 10 (keep other hyperparameters as default values) and fit it on the training set


# TO-DO: Use the model to predict on the test set and save these predictions as `y_pred`


# TO-DO: Find the R-squared score and store the value in `ridge_score`


In [ ]:
# Grader (2 points)
grader.grade(test_case_id = 'check_ridge_reg', answer = (ridge_score, reg_ridge.alpha))

_**Exam-Style Practice Questions:**_

_(While we will NOT grade this for the purpose of the HW, it is in your best interest to take just 30-60 seconds to think about this as I may very well put something similar in the final exam!_ 😉_)_

- What do you notice about the $R^2$ scores of the Ridge and unregularized Linear Regression above?
- If Ridge is supposed to "improve" Linear Regression, we may find it unusual if that is not happening here. Under what circumstances would we consider using Ridge over unregularized Linear Regression?
- Would you expect different results if we made `alpha` bigger ($\alpha \rightarrow \infty$)? smaller ($\alpha \rightarrow 0$)? Why or why not?

#### (c) Random Forest Regression [3 pts]

From the results of linear regression and ridge linear regression, we see a linear model may not be ideal for this regression problem. We want a model that can capture the complex relationship between the target variable and the features. Fortunately, we have access to ensemble methods, which combine multiple simple models to improve performance. We will explore the bagging algorithm called Random Forest Regressor using scikit-learn.

Use the `RandomForestRegressor` class in Scikit-learn's ensemble library to perform Random Forest Regression. Initialize a Random Forest regression model named `reg_rf`, fit the model to the training set, and then make predictions on the testing set.

Report your R-squared score and save it as a variable called `rfr_score`, set random state to 42

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Initialize model with default parameters and fit it on the training set


# TO-DO: Use the model to predict on the test set and save these predictions as `y_pred`


# TO-DO: Find the R-squared score and store the value in `rfr_score`


In [ ]:
# Grader (3 points)

grader.grade(test_case_id = 'check_rf_reg', answer = (rfr_score, reg_rf.feature_names_in_, reg_rf.n_outputs_, y_pred))

### **1.4.4** K-Means Clustering

K-means clustering is a popular unsupervised machine learning algorithm that is used for grouping similar data points into a predetermined number of clusters. It is a simple and effective algorithm that is widely used in various fields such as image processing, recommendation systems, and customer segmentation.

An elbow plot is a visualization tool used to determine the optimal number of clusters for a dataset. It plots the within-cluster sum of squares (WCSS) against the number of clusters. WCSS is the sum of the squared distance between each data point and its assigned centroid. The elbow plot helps you to choose the number of clusters that minimize the WCSS while avoiding overfitting.

Please refer to [this document](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) to find out how to compute the sum of squared distances

**We will be using the same features from the <ins>classification</ins> setting.
To determine the best $k$ for the clustering, let's firstly generate an elbow plot.**
- Consider number of clusters from 2 to 10
- Set parameter `n_init` to 5.
- Set random seed to 0

**The elbow plot (4 points) will be manully graded**
- x-axis is the "number of clusters", i.e. "k".
- y-axis is the within-cluster sum of squares (WCSS) value
- add proper titles for the plot and axis.

Note that standardizing the data before performing k-means clustering is often recommended, but for illustration purpose, we do not standardize the input in this section.

#### (a) Find the best number of clusters with the elbow plot [4 + 3 pts]

In [ ]:
# TO-DO: Prepare the data (the same features from the classification setting, i.e. excluding "booking_status")
# stores features dataframe into variable called "features"


In [ ]:
# Import the necessary libraries


# TO-DO: [MANUALLY GRADED: 4 points]
#   for k ranges from 2 to 10, fit on "features" to generate an elbow plot


In [ ]:
# TO-DO: choose the best number of clusters (the elbow) and fill in
number_of_cluster =

# TO-DO: fill in the sum of squared distances for the best number of clusters
wcss_elbow =

In [ ]:
# Grader (3 points)
grader.grade(test_case_id = 'kmeans_elbow', answer = (number_of_cluster, wcss_elbow))

#### (b) Re-fit with the best number of clusters [3 pts]

In [ ]:
# TO-DO: re-run the K-Means clustering with the best number of clusters, save the fitted model to `kmeans`
kmeans =

In [ ]:
# Grader (3 points)
grader.grade(test_case_id = 'refit_kmeans', answer = (Counter(kmeans.labels_), kmeans.n_features_in_))

# **Part II:** Distributed Machine Learning with Spark (35  points)

Apache Spark ML is a machine learning library that consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering, dimensionality reduction, and underlying optimization primitives.

**Why Spark ML?**

Moving to the Big Data Era requires heavy iterative computations on very large datasets. Standard implementations of machine learning algorithms require very powerful machines to be able to run. However, depending on high-end machines is not advantageous due to their high price and improper costs of scaling up. The idea of using distributed computing engines is to distribute the calculations to multiple low-end machines (commodity hardware) instead of a single high-end one. _This **definitely speeds up** the learning phase and allows us to create better models._

Read more about it with the python documentation **[here](https://spark.apache.org/docs/latest/ml-guide.html)**.

Run the code below to set up Spark.

## _PENALTY WARNING_:
We will very carefully go through the code in your notebook for this HW. Any student caught trying anything devious like using sklearn to avoid using SparkML in this section will get a **ZERO**.

**Note:** The cell below may take upto **4-5 minutes** to run so be patient!

In [ ]:
%%capture

!apt install libkrb5-dev
!wget https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install findspark
!pip install sparkmagic
!pip install pyspark
! pip install pyspark --user
! pip install seaborn --user
! pip install plotly --user
! pip install imageio --user
! pip install folium --user

In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder.appName('bigdata-hw4').getOrCreate()
sqlContext = SQLContext(spark)

In [ ]:
%load_ext sparkmagic.magics

In [ ]:
# Graph section
import networkx as nx

# SQLite RDBMS
import sqlite3

import os
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop3.2'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

## **2.1** Initializing Spark Data [Total: 2 Points]

We have the Spark setup ready, and we now need the data for our ML algorithms. We will use the data you processed in Part I, but in Spark.

### **2.1.0** Converting the Pandas Dataframe into a Spark Dataframe

Read the `encoded_df_reservations` that you created in Part I into a Spark dataframe (`sdf`) and name the SDF as `reservations_sdf`. \\
**Note:** Just like Part I, the target variable (label) for this section will be `"booking_status"`.

In [ ]:
#TO-DO: Read encoded_df_reservations into a Spark Dataframe called reservations_sdf


Print out the Dataframe Schema and verify the datatypes. If you did everything correctly so far, the schema should be:
```
root
 |-- no_of_adults: long (nullable = true)
 |-- no_of_children: long (nullable = true)
 |-- no_of_weekend_nights: long (nullable = true)
 |-- no_of_week_nights: long (nullable = true)
 |-- required_car_parking_space: long (nullable = true)
 |-- lead_time: long (nullable = true)
 |-- arrival_year: long (nullable = true)
 |-- arrival_month: long (nullable = true)
 |-- arrival_date: long (nullable = true)
 |-- repeated_guest: long (nullable = true)
 |-- no_of_previous_cancellations: long (nullable = true)
 |-- no_of_previous_bookings_not_canceled: long (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: long (nullable = true)
 |-- booking_status: double (nullable = true)
 |-- type_of_meal_plan_Meal Plan 1: long (nullable = true)
 |-- type_of_meal_plan_Meal Plan 2: long (nullable = true)
 |-- type_of_meal_plan_Meal Plan 3: long (nullable = true)
 |-- type_of_meal_plan_Not Selected: long (nullable = true)
 |-- room_type_reserved_Room_Type 1: long (nullable = true)
 |-- room_type_reserved_Room_Type 2: long (nullable = true)
 |-- room_type_reserved_Room_Type 3: long (nullable = true)
 |-- room_type_reserved_Room_Type 4: long (nullable = true)
 |-- room_type_reserved_Room_Type 5: long (nullable = true)
 |-- room_type_reserved_Room_Type 6: long (nullable = true)
 |-- room_type_reserved_Room_Type 7: long (nullable = true)
 |-- market_segment_type_Aviation: long (nullable = true)
 |-- market_segment_type_Complementary: long (nullable = true)
 |-- market_segment_type_Corporate: long (nullable = true)
 |-- market_segment_type_Offline: long (nullable = true)
 |-- market_segment_type_Online: long (nullable = true)
```

**Note:** Especially ensure that the target variable is of the datatype `double`. Cast it to the correct datatype, if needed.

In [ ]:
# TO-DO: Verify that the schema of your Spark dataframe matches the above
#        Make sure to double-check that the target variable i.e. booking_status is of type "double"


### **2.1.1** Setting Up a VectorAssembler

Great! We have the processed data now. For Spark ML, we need to create a feature column which has all features concatenated as a list, and a single column for labels (which we already have!)
We will use `VectorAssembler()` to create a feature vector from all our features, and we will call this vectorized variable as “features”.

First, list all column names in `reservations_sdf` and store them in a list variable called `all_columns`.

In [ ]:
# TO-DO: store all column name in the data_sdf in a list called "all_columns"


Create a list of column(s) you don't want to include as your features. Name this list `drop_columns`.

In [ ]:
# TO-DO: Create "drop_columns"


Now, using `drop_columns`, create a list called `feature_columns` that only contains the names of the columns that would be used as features for our ML models.

In [ ]:
# TO-DO: Create the "feature_columns" list


In [ ]:
# Grader cell [2 points]
grader.grade(test_case_id = 'check_feature_columns', answer = feature_columns)

Finally, create a [VectorAssembler](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html) object with the columns that you want to use as features. Name your output column as `features` (i.e. they are the features that will be used for SparkML).

In [ ]:
# TO-DO: Import required library and create VectorAssembler object (to be used later)


## **2.2** Preprocessing: Pipeline and Train-Test Split [Total: 3 Points]

Now that we have a VectorAssembler object ready, let's use it on our Spark dataframe to get the concatenated features column. To do this, we implement a [Pipeline](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.Pipeline.html), which can be used to streamline multiple preprocessing stages at once. For this data, we just need a single stage with the assembler, but you could have other stages before that where you perform operations on the data like converting categorical strings in the features to numeric values, or do feature scaling operations.

We will create a pipeline with a single stage — the assembler. Fit the pipeline on `reservations_sdf` to create the transformed dataframe and name it `processed_reservations_sdf`.


In [ ]:
# TO-DO: Import required libraries


# TO-DO: Define a pipeline object


# TO-DO: Fit and transform the pipeline on the data and store the transformed sdf as "processed_reservations_sdf"


Now that we have the data in the format we need, we will create our train and test sets.

Conduct a train-test split where 80% of the data is assigned to the training set while the remaining 20% is assigned to the testing set.

- Name these sets as `train_sdf` and `test_sdf` respectively.
- Set the variable `random_seed` to 42 and then add the argument `seed = random_seed` in the function to fix the random state in order to ensure consistency with our results.

Note: Fit the pipeline on reservations_sdf

In [ ]:
# TO-DO: Assign appropriate value to the random_seed variable


# TO-DO: Do 80/20 train-test split with seed = random_seed and store them as "train_sdf" and "test_sdf"


In [ ]:
# Grader (3 points)
grader.grade(test_case_id = 'check_train_test_split_spark', answer = (train_sdf.count(), test_sdf.count(), random_seed))

## **2.3** Modeling (SparkML) [Total: 30 Points]

Time to do the cool stuff! We will work on various types of models using SparkML to see the difference in implementation from sklearn.

### **2.3.1** Vanilla Logistic Regression [6 pts]

Let's train an **unregularized** Logistic Regression model to our data and try to predict whether the `booking_status` will be cancelled or not.

This time, we will use "Big" data tools to help us build a Logistic Regression model for binary classification. Look up the documentation online and try to understand how to implement this.


**Note**: Make sure to set the parameter `maxIter` to an appropriate value (e.g. 5) for fast computation.

**Checks:**

- Are you sure you're using the correct evaluation metric? Always read the Official Documentation to see if the default parameters are what you expect. This is a recurring theme for many issues.

- Did you make sure to order the inputs to the confusionMatrix function correctly so that it returns the confusion matrix in the form [[TP, FN], [FP, TN]]

- Have you made sure to convert the booking_status column to type double? If you are facing a Py4JJavaErrors error, be sure to do so.

In [ ]:
# TO-DO: Import required library for Logistic Regression


# TO-DO: Instantiate and fit Logistic Regression model to training data



Calculate the training accuracy using the model's summary and store it in a variable called `train_accuracy`.

In [ ]:
# TO-DO: Get training accuracy and store it as `train_accuracy`


Now, let's find out how good the model actually is and see if it overfits to the training data.

Predict the labels `"booking_status"` for your test data and store them as `predictions` _(Hint: it is called 'transform' in SparkML)_

In [ ]:
# TO-DO: Make predictions on testing set and store it as "predictions"


To get the test accuracy, we will make use of a **confusion matrix**. It is used to consolidate the predictive performance of a model into a single table. In a binary classification scenario, it looks like this:
<img src = "https://s2.loli.net/2023/04/01/Bg4yaIRGkEZXxvD.png">

Given the confusion matrix, we can calculate various metrics such as accuracy, precision, recall, F1 score etc.

For our task, we will calculate the test accuracy. The formula for this is: 

> Accuracy = $\frac{TP + TN}{TP + FP + TN + FN}$

Evaluate the performance using the Confusion Matrix using [MulticlassMetrics](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.evaluation.MulticlassMetrics.html) object in the SparkML Regression library. Store the Confusion Matrix as an numpy array named  `confusion_matrix`. \\

(*Ignore any warnings you may get*)

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Select appropriate columns to use MulticlassMetrics


# TO-DO: Instantiate metrics objects


# TO-DO: Create confusion matrix and store it as a numpy array named "confusion_matrix"


Now, calculate the test accuracy using the Confusion Matrix obtained above and store it in a variable called `test_accuracy`.

In [ ]:
# TO-DO: Calculate test accuracy using the confusion matrix and store it as "test_accuracy"


In [ ]:
# Grader cell (6 points)
grader.grade(test_case_id = 'check_unreg_logistic', answer = (train_accuracy, test_accuracy, confusion_matrix))

### **2.3.2** Regularized Logistic Regression [8 pts]

Now, we will add regularization – LASSO (L1), Ridge (L2) and elastic net (combination of L1 and L2), to avoid overfitting. You can play around with different regularization hyperparameters when initializing the 3 different regularized logistic regression models.

`elasticNetParam` controls the type of parameterisation, which when at 0 and 1 denotes ridge regression and LASSO regression respectively and everything between a linear combination of both. `regParam` controls the degree of parametrisation (defaults to 1.0). More mathematical details can be found here: https://runawayhorse001.github.io/LearningApacheSpark/reg.html. (Hint: choose your `regParam` wisely)

Compare these with each other and with the unregularized regression performed in 2.3.1.

Make sure to evaluate the performance on test data using the same methodology as above (using a confusion matrix to calculate the accuracy).

**Note**: Just like 2.3.1, set `maxIter` to a suitable value.


#### (a) LASSO (L1)

First, implement a LASSO Logistic Regression model using SparkML and call it `l1_model`, fit it on `train_sdf` and get predictions using `test_sdf`. Get the training accuracy and store it as `l1_train_accuracy`.

In [ ]:
# TO-DO: Instantiate LASSO/L1 regularized model as "l1_model"


# TO-DO: Fit and Transform using "l1_model"


# TO-DO: Store training accuracy as "l1_train_accuracy"


From the predictions, create a confusion matrix and use it to get the test accuracy. Store this accuracy value as `l1_test_accuracy`.

In [ ]:
# TO-DO: Create the Confusion matrix


# TO-DO: Calculate the test accuracy as done in the previous section. Store test accuracy as "l1_test_accuracy"


#### (b) Ridge (L2)

Next, implement a Ridge Logistic Regression model using SparkML and call it `l2_model`, fit it on `train_sdf` and get predictions using `test_sdf`. Get the training accuracy and store it as `l2_train_accuracy`.

In [ ]:
# TO-DO: Instantiate Ridge/L2 regularized model as "l2_model"


# TO-DO: Fit and Transform using "l2_model"


# TO-DO: Store training accuracy as "l2_train_accuracy"


From the predictions, create a confusion matrix and use it to get the test accuracy. Store this accuracy value as `l2_test_accuracy`.

In [ ]:
# TO-DO: Create the Confusion matrix


# TO-DO: Calculate the test accuracy as done in the previous section. Store test accuracy as "l2_test_accuracy"


#### (c) Elastic Net

Finally, implement an Elastic Net Logistic Regression model using SparkML and call it `en_model`, fit it on `train_sdf` and get predictions using `test_sdf`. Get the training accuracy and store it as `en_train_accuracy`.

In [ ]:
# TO-DO: Instantiate Elastic Net regularized model as "en_model"


# TO-DO: Fit and Transform using "en_model"


# TO-DO: Store training accuracy as "en_train_accuracy"


From the predictions, create a confusion matrix and use it to get the test accuracy. Store this accuracy value as `en_test_accuracy`.

In [ ]:
# TO-DO: Create the Confusion matrix


# TO-DO: Calculate the test accuracy as done in the previous section. Store test accuracy as "en_test_accuracy"


In [ ]:
#8 points
#Grader cell
grader.grade(test_case_id = 'check_reg_logistic', answer = (l1_model.getElasticNetParam(), l1_model.getRegParam(), l1_train_accuracy, l1_test_accuracy,
                                                            l2_model.getElasticNetParam(), l2_model.getRegParam(), l2_train_accuracy, l2_test_accuracy,
                                                            en_model.getElasticNetParam(), en_model.getRegParam(), en_train_accuracy, en_test_accuracy))

### **2.3.3** Random Forest Classification [8 pts]

If you are looking to win Kaggle competitions, you definitely must know about Random Forests, Boosted Trees, etc. These ensemble methods generalize and work surprisingly well for a lot of classification problems and sometimes for regression problems (though regression trees are not the focus of this course).

Let's give it a go. Similar to what you did for the Logistic Regression model, create a Random Forest classifier model and name it `rf` and fit it to the training data. Call the fitted model `rf_model`.

**Note1:** Set the `random_seed` to 42 and set the argument `seed = random_seed` while creating `rf` as a RandomForest object. \\
**Note2:** Set `maxDepth` to a reasonable value (e.g. 10) to find a good balance between performance and computation time.

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Set random_seed to 42


# TO-DO: Instantiate the RF Model and call it "rf", then fit it on the training data


This time, get predictions on both the training and testing sets, and store them as `train_pred` and `test_pred` respectively.

In [ ]:
# TO-DO: Get predictions and save to "train_pred" and "test_pred" respectively


Evaluate using the accuracy metric again, similar to the previous sections.
- Store the confusion matrices as `rf_train_cm` and `rf_test_cm` respectively. Note that the accuracies should be manually computed based on your confusion matrix.
- Store the training and test accuracy as `rf_train_accuracy` and  `rf_test_accuracy` respectively.

In [ ]:
# TO-DO: Evaluate the prediction accuracy for train set and call it "rf_train_accuracy"


# TO-DO: Evaluate the prediction accuracy for test set and call it "rf_test_accuracy"


In [ ]:
# Grader (8 points)
grader.grade(test_case_id = 'check_random_forest', answer = (rf_train_accuracy, train_pred.count(), rf_test_accuracy, test_pred.count(), rf_train_cm, rf_test_cm))

### **2.3.4** Dimensionality Reduction Using PCA [8 pts]

We will again use the powerful PCA to reduce the dimensions and project the data onto a lower dimensional space and fit a logistic regression on the new projected data.

Initialize a PCA model on SparkML, where you select an appropriate number of dimensions, and call it `pca`. Then, fit the model on the training data and get the PCA features from the trained model.

**Hint**: Choose `k` for the PCA model using the explained variance plot you made earlier.

**Note:**

- Notice we're in SparkML world here, not sklearn! Check the Official Documentation for SparkML's StandardScaler to verify if the default parameters are what we would expect.

- Also note that SparkML's scaling works slightly differs from sklearn's. The following link should help show and explain why we needed to configure withMean and withStd parameters a certain way for SparkML.

- Make sure you are passing the correct inputs for Logistic Regression.

- You may refer to these Stack Overflow posts: [[Link 1]](https://stackoverflow.com/questions/47770674/pca-output-in-spark-doesnt-matches-with-scikit-learn)[[Link 2]](https://stackoverflow.com/questions/57349987/standardscaler-difference-between-with-std-false-or-true-and-with-mean-false) highlighting the differences between using PCA and Standard Scaler in Scikit-learn vs. PySpark ML.

In [ ]:
# TO-DO: Import required libraries


# TO-DO: Perform intermediate steps to address scale-variance in PCA


# TO-DO: Instantiate PCA object as "pca" and then fit and transform.
#        Make sure you make a reasonable choice for the number of Principal Components


Now, create a Logistic Regression model and train it using the PCA features.

In [ ]:
# TO-DO: Instantiate Logistic Regression model and call the model object "lr_model"


# TO-DO: Fit Logistic Regression Model and get predictions


Calculate the training accuracy using the model's summary and store it in a variable called `train_accuracy_pca`.

In [ ]:
# TO-DO: Get training accuracy and store it as `train_accuracy_pca`



Evaulate the test performance by creating a confusion matrix and call it `confusion_matrix_pca`. Then, calculate the test accuracy using the confusion matrix  and store it as `test_accuracy_pca`.

In [ ]:
# TO-DO: Create confusion matrix and store it as "confusion_matrix_pca"


# TO-DO: Calculate test accuracy and store it as "test_accuracy_pca"


In [ ]:
# Grader cell (8 points)
grader.grade(test_case_id = 'check_pca', answer = (pca.getK(), len(lr_model.coefficients), train_accuracy_pca, test_accuracy_pca, confusion_matrix_pca))

# Homework Submission Reminders & Penalties

Good job! You have finished the homework. The submission instructions are as follows:

* **Double check** that you have the correct PennID (all numbers) in the autograder.

* **Triple check** that you have all plots shown in this Colab notebook before submitting (otherwise, your submission will be subjected to a **penalty of -5 points**).

*  Go to the "File" tab at the top left of the Colab UI, click "Download .ipynb" and then "Download .py".  **Rename** these two files to `homework4.ipynb` and `homework4.py` respectively and upload them to Gradescope.
  - <ins>**WAIT UNTIL THE GRADESCOPE AUTOGRADER FINISHES RUNNING!**</ins> If we have to manually upload this for you after the deadline, **we will apply a penalty of -10 points** (this stacks with any other penatlies)

* After the autograder finishes running, Remember to click on the "Code" tab and see what your `.ipynb` file looks like on Gradescope.
  - Just like in HW2, if "Large File Hidden" shows up, you MUST fix it or ensure that you submit screenshots of your plots along with the code cell that generated it

* Post any issues with submission on Ed and make sure to keep in mind the late day policy.

* After you submit your code, the teaching staff will manually grade your Colab notebook in order to validate the correctness of your code.